# Application d'extractions des relations sociales entre entités nommées

- Trouver d'autres construction grammaticales qui peuvent décrire des relations et améliorer make_relation
- Penser aux words-embedding

In [47]:
from stanfordcorenlp import StanfordCoreNLP
from nltk.tokenize import sent_tokenize, word_tokenize
import re

In [48]:
nlp = StanfordCoreNLP('stanford-corenlp-full-2018-10-05')

In [49]:
def extract_NE(text,PERSON):
    """ Extrait les entités nommées d'un texte (text) et les stocke dans une liste (PERSON)."""
    NE = nlp.ner(text)
    for elt in NE :
        if 'PERSON' in elt :
            PERSON.append(elt[0])

    return(list(set(PERSON)))
    
def init_relationship(RELATIONSHIP,PERSON) :
    """ Initialise un dictionnaire (RELATIONSHIP) sur la base d'une liste (PERSON) d'entité nommée. """
    for p in PERSON : 
        RELATIONSHIP[p] = []

def extract_dependencies(text_tokens,DEPENDENCY) :
    """
    Extrait les dépendances 'nmod:poss' et 'appos' au sein d'un texte découpé par phrases (text_tokens) et les insère dans DEPENDENCY.
    
    arguments : 
    
    text_tokens : liste contenant des listes, chaque 'sous-liste' contient elle une phrase du texte d'origine.
    DEPENDENCY : tableau qui contient dans chaque colonne les dépendances grammaticales liées à une phrase.
    
    """
    nb_line = 0
    for line in text_tokens  :
        parsing = nlp.dependency_parse(line)
        for elt in parsing:

            if ('nmod:poss' in elt or 'appos' in elt or 'nmod' in elt or 'compound' in elt) :
                DEPENDENCY[nb_line] = DEPENDENCY[nb_line] + [(elt)]
                
        nb_line+=1
    nlp.close()
    
def make_relation(dep,dependencies,nb_line):
    """ 
    Crée un quadruplé qui correspond à une relation valide sous la forme (personne1,relation,personne2,ligne du texte
    où la relation a été identifiée).
    
        Arguments :
        dep : une dépendance de la forme ('nature de la dépendance',mot1,mot2)
        dependencies : liste de dépendances contenues dans la même ligne de texte que 'dep'
        nb_line : numéro de la ligne du texte où la dépendance a été récupérée
        
    """
    #On vérifie que la dépendance est bien de type 'nmod:poss'
    if(dep[0]=='nmod:poss' or dep[0]=='nmod') :
        
        # Si il n'y a pas d'autres dépendance dans la phrase, alors on ne pourra pas extraire les 2 personnages
        # On renvoie donc un quadruplé de type ('NaN',relation,personnage2,numéro de la ligne dans le texte)
        if (dependencies == []):
            return(('NaN',dep[1],dep[2],nb_line))
        
        #Sinon...
        else :
            #..on va chercher dans le reste des dépendances...
            for elt in dependencies :
                #..une apposition.
                #Si l'élement (elt[1]) (qui est un personnage) de l'apposition est identique à celui de notre nmod:poss (dep[1])...
                #..cela signifie qu'ils sont liés par un élément commun (qui est la relation dans le texte)
                if ((elt[0]=='appos' and dep[0]=='nmod:poss')and dep[1]==elt[1] ):
                    #Alors on retourne le quadruplé (personnage1,relation,personnage2,numéro de la ligne dans le texte)
                    return((elt[2],dep[1],dep[2],nb_line))
                #La même chose mais pour un 'nmod'
                elif ((elt[0]=='appos' and dep[0]=='nmod')and dep[1]==elt[2] ):
                    return((elt[1],dep[1],dep[2],nb_line))
                
                elif ((elt[0]=='compound' and dep[0]=='nmod:poss') and elt[1]==dep[1]):
                    return((dep[2],elt[2],dep[1],nb_line))
                    
        
    return((0,0,0))

def replace_by_NE(text_split, PERSON):
    """Remplace toutes les occurences de la liste 'to_replace' par l'entité nommée à laquelle
    elles font référence dans le texte """
    
    to_replace = ['he','she','his','him','her','He','She','His','Her','Him']
    result = ""
    tmp_NE = ""
    
    for elt in text_split :
        
        tmp = re.sub('\W+','',elt)
        
        if(tmp in PERSON):
            tmp_NE = tmp 
        
        if(elt in to_replace) :
            
            #Ici on rajoute le suffixe "'s" quand on rencontre 'his' ou 'her' car sinon on risque de changer les dépendances grammaticales
            #Et l'analyseur de dépendance risque de ne pas détecter un nmod:poss ou une apposition
            
            if(elt=="his" or elt=="her") :
                result = result + tmp_NE+"'s'" + " "
            else :
                result = result  + tmp_NE + " "
        
        else : 
            result = result + elt + " "
            
    return result


    
def fill_relationship(DEPENDENCY,TOKENS,LINKS,RELATIONSHIP,INCOMPLETE_RELATIONSHIP):
    
    """
    Remplis le dictionnaire RELATIONSHIP avec les relations qui ont pu être extraites à l'aide de la fonction
    'make_relation()'
    
    arg : 
    
    DEPENDENCY : Le tableau de dépendances dont on a parlé dans les fonctions précédentes
    
    TOKENS : Une liste de liste. Chaque sous-liste contient une phrase du texte qui a été découpée en 'token' dont
    les indices des mots correspondent à ceux présents dans les dépendances du tableau DEPENDENCY. Cette liste va
    nous permettre de transformer un indice en mot pour pouvoir avoir une sortie compréhensible.
    
    RELATIONSHIP : Dans la description de la fonction
    
    LINKS : Liste qui contient les relations possibles entre des personnages
    """
    nb_line = 0
    tmp = []
    encoded_relationship = []
    id_NaN = 0
    
    #Pour toutes les dépendances qu'on a recueilli dans le tableau DEPENDENCY on crée les relations qui existent
    #en utilisant la fonction make_relation et on les stocke dans la liste tmp
    for elt in DEPENDENCY:
        try:
            for i in elt :
                tmp.append(make_relation(i, elt, nb_line))
            nb_line += 1

        except:
            nb_line += 1

    #Pour chaque élément de la liste tmp, on remplit la liste "encoded_relationship" de tous les éléments validés
    for t in tmp : 
        if(t!=(0,0,0)) :
            encoded_relationship.append(t)
    
    #Cette boucle va permettre de traduire une relation de type (1,mother,3,5) en (Gertrude,mother,Hamlet) et la stocker
    #dans le dictionnaire RELATIONSHIP
    for r in encoded_relationship :

        try :
            line = r[3]
            perso2 = r[0]-1
            lien = r[1]-1
            perso1 = r[2]-1


            try :

                if(((TOKENS[line][lien]) in LINKS and (TOKENS[line][perso1]) in PERSON) and TOKENS[line][perso2] in PERSON ) :
                    RELATIONSHIP[(TOKENS[line][perso1])] += [(TOKENS[line][lien],TOKENS[line][perso2])]

            except :

                if(((TOKENS[line][lien]) in LINKS and (TOKENS[line][perso1]) in PERSON) and TOKENS[line][perso2] in PERSON) :
                    RELATIONSHIP[(TOKENS[line][perso1])] = [(TOKENS[line][lien],TOKENS[line][perso2])]

        #S'il manque un élément dans la relation, on le stocke dans le dictionnaire correspondant
        except :

            line = r[3]
            lien = r[1]-1
            perso1 = r[2]-1

            try :
                if((TOKENS[line][lien]) in LINKS) :
                    INCOMPLETE_RELATIONSHIP[TOKENS[line][perso1]] += [(TOKENS[line][lien],str(r[0])+str(id_NaN))]
            
                    id_NaN +=1
            except : 
                
                if((TOKENS[line][lien]) in LINKS) :
                    INCOMPLETE_RELATIONSHIP[TOKENS[line][perso1]] = [(TOKENS[line][lien],str(r[0])+str(id_NaN))]
            
                    id_NaN +=1
                

def make_correspondance(RELATIONSHIP,LINK_CORRESPONDANCE):
    """
    Créer un dictionnaire dans lequel on trouve la symétrie de chaque relation présente dans le dictionnaire RELATIONSHIP
    
    Par exemple : 
    RELATIONSHIP : Hamlet : [(mother,Gertrude)] -> new_RELATIONSHIP : Gertrude : [(child,Hamlet)]
    (Hamlet a pour mère Gertrude) -> (Gertrude a pour fils Hamlet)
    
    """
    new_RELATIONSHIP = {}
    
    for elt in RELATIONSHIP : 
        for i in RELATIONSHIP[elt] :

            try :
                new_RELATIONSHIP[i[1]] += [(LINK_CORRESPONDANCE[i[0]],elt)]
                
            except : 

                new_RELATIONSHIP[i[1]] = [(LINK_CORRESPONDANCE[i[0]],elt)]
    return new_RELATIONSHIP

def merge_dictionnary(dict1,dict2) : 
    """Fusionne deux dictionnaire (va nous permettre de fusionner RELATIONSHIP et son symétrique)"""
    for elt in dict2 :
        for i in dict2[elt] :
            try :
                dict1[elt] += [i]
            except : 
                dict1[elt] = [i]

In [50]:
file = open('Hamlet.txt','r+')
text = file.read()
text = text.replace("’\n","\n")
PERSON = []
DEPENDENCY = [[]]*len(text)
TOKENS = []
RELATIONSHIP = {}
INCOMPLETE_RELATIONSHIP = {}
LINKS = ['son','father','mother','daughter','cousin','siblings','husband','wife','mate','spouses','brother','sister','friend','girlfriend','boyfriend', 'uncle','aunt','nephew','niece']
LINK_CORRESPONDANCE = {
    'son' : 'parent',
    'father' : 'child',
    'mother' : 'child',
    'daughter' : 'parent',
    'cousin' : 'cousin',
    'siblings' : 'siblings',
    'husband' : 'wife',
    'wife' : 'husband',
    'spouses' : 'spouses',
    'brother' : 'siblings',
    'sister' : 'siblings',
    'friend' : 'friend',
    'girlfriend' : 'couple',
    'boyfriend' : 'couple',
    'uncle' : 'nephew/niece',
    'aunt' : 'nephew/niece',
    'nephew' : 'uncle/aunt',
    'niece' : 'uncle/aunt',
    'mate' : 'mate'
}

In [ ]:
PERSON = extract_NE(text,PERSON)
init_relationship(RELATIONSHIP,PERSON)

print(RELATIONSHIP)

In [ ]:
tmp_text = text.split()
text = replace_by_NE(tmp_text,PERSON)
text_tokens = sent_tokenize(text)

for line in text_tokens : 
    TOKENS.append(nlp.word_tokenize(line))

In [ ]:
parsing = nlp.dependency_parse(text)        
extract_dependencies(text_tokens,DEPENDENCY)

In [ ]:
fill_relationship(DEPENDENCY,TOKENS,LINKS,RELATIONSHIP, INCOMPLETE_RELATIONSHIP)

In [ ]:
corresponding_RELATIONSHIP = make_correspondance(RELATIONSHIP,LINK_CORRESPONDANCE)
corresponding_INCOMPLETE_RELATIONSHIP = make_correspondance(INCOMPLETE_RELATIONSHIP,LINK_CORRESPONDANCE)

In [ ]:
merge_dictionnary(RELATIONSHIP,corresponding_RELATIONSHIP)
merge_dictionnary(INCOMPLETE_RELATIONSHIP,corresponding_INCOMPLETE_RELATIONSHIP)
print(INCOMPLETE_RELATIONSHIP)

In [ ]:
for elt in RELATIONSHIP :
    print(elt,' :', RELATIONSHIP[elt])

Problèmes rencontrés :

TOKENIZATION : 

- On a utilisé la tokenization par phrase de la librairie, c'est à dire qu'on a découpé le texte par phrase avec une liste qui contient dans chacune de ses cases une phrase du texte.

    - Le problème : La fonction n'a pas toujours découpé le texte par phrase (voir par exemple la case numéro 5 dans laquelle on retrouve 2 phrases)



- On a, pour chacune des cases, utilisé l'analyseur de dépendences qui nous permet d'avoir les relations nmod:pos et appos entre les mots pour chacune des phrases qu'on a tokénizé.

    - Le problème est que l'analyseur de dépendences, lui, découpe correctement les phrases. Ainsi, lorsqu'il nous donne l'indice d'un mot, il se base sur son découpage et non celui de la fonction qu'on a utilisé pour découper nos phrases, ce faisant, les indices des mots ne correspondent plus entre notre phrase du texte tokénisé et notre phrase du texte analysé.


Solution temporaire : 

Le problème vient des caractères spéciaux "’" qui encadrent les dialogues, il suffit de les supprimer pour ce texte précis.


STYLE D'ECRITURE : 

Notre fonction 'make_relation' s'appuie sur une reconnaissance des relations lorsqu'elles sont écrites à l'aide d'une construction grammaticale simple et basique. Il y'a autant de manière de reconnaitre une relation à l'aide de construction grammaticales qu'il y'a de manière de décrire une relation. Ainsi, certains textes plus élaboré posent problème et il faut ajouter des règles à la fonction "make_relation".

REMPLACEMENT DES 'HIS/HER/HE/SHE/etc.' à optimiser car fonctionne mal

LA RECONNAISSANCE DES ENTITES NOMMEES COMME SCAR MARCHE PAS TOUJOURS

Solution : Peut être réglé avec le nmod

FAUTES DE FRAPPE DANS LE TEXTE.